In [2]:
import os
import cv2 as cv
import numpy as np 
import pandas as pd

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential , load_model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical

In [ ]:
###################################################### 실행 안함. 자동완성용

from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import to_categorical
######################################################

In [4]:

folder_list = os.listdir('/Users/jsp/Desktop/workspace/projects/fstteamproject/main/git_teamprj/proj1/open/train')
folder_list.remove('.DS_Store') 

data = []
pic_count = []

for folder_name in folder_list:
    folder_path = '/Users/jsp/Desktop/workspace/projects/fstteamproject/main/git_teamprj/proj1/open/train/' + folder_name
  
    image_list = os.listdir(folder_path)
    list_len = len(image_list)
    pic_count.append(list_len)
    
    for image_name in image_list:
        image_path = folder_path +'/' + image_name
        img = cv.imread(image_path)
        resize = cv.resize(img, (400,400), interpolation=cv.INTER_CUBIC)
        gray = cv.cvtColor(resize, cv.COLOR_BGR2GRAY)
        data.append(gray)

In [7]:
answer = []
for idx, nm in enumerate(folder_list):
        answer += [nm] * pic_count[idx]
answer = np.array(answer)
answer = pd.get_dummies(answer)
answer = answer.values

In [8]:
answer


array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ...,  True, False, False],
       [False, False, False, ...,  True, False, False],
       [False, False, False, ...,  True, False, False]])

In [9]:
xtrain, xtest, ytrain, ytest = train_test_split(np.array(data),answer,test_size=0.2, shuffle=True)

In [10]:
print(xtrain.shape)
print(xtest.shape)

(2765, 400, 400)
(692, 400, 400)


In [11]:
xtrain = xtrain.reshape(2765,400,400,1).astype('float32')/255
xtest = xtest.reshape(692,400,400,1).astype('float32')/255

In [18]:
model = Sequential()
model.add(Conv2D(32, (3,3), input_shape =(400,400,1), activation='leaky_relu'))
model.add(Conv2D(64, (3,3), activation='leaky_relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64, activation= 'leaky_relu'))
model.add(Dropout(0.5))
model.add(Dense(19, activation= 'softmax'))

model.summary()
model.compile(optimizer= 'adam', loss='categorical_crossentropy',metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 398, 398, 32)      320       
                                                                 
 conv2d_5 (Conv2D)           (None, 396, 396, 64)      18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 198, 198, 64)     0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 198, 198, 64)      0         
                                                                 
 flatten_2 (Flatten)         (None, 2509056)           0         
                                                                 
 dense_5 (Dense)             (None, 64)                160579648 
                                                      

In [19]:
modelpath="bestmodel.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=0, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=20)

In [20]:
history = model.fit(xtrain,ytrain, validation_data=(xtest, ytest), batch_size=50, epochs = 500, verbose=1, callbacks=[early_stopping_callback,checkpointer])

Epoch 1/500
56/56 [==============================] - 223s 4s/step - loss: 17.0024 - accuracy: 0.1942 - val_loss: 2.9359 - val_accuracy: 0.4032
Epoch 2/500
31/56 [===============>..............] - ETA: 1:30 - loss: 3.0302 - accuracy: 0.2897

In [18]:
my_model = load_model('bestmodel.hdf5')

In [19]:
score = my_model.evaluate(xtest, ytest)
print('Test accuracy:', score[1])

22/22 [==============================] - 1s 39ms/step - loss: 2.9381 - accuracy: 0.4104
Test accuracy: 0.41040462255477905
